In [14]:
#data cleaning
import pandas as pd
import numpy as np
fighter_info_final = pd.read_csv("fighter_df.csv")

In [15]:
to_feed = pd.read_csv("we_good.csv")

In [16]:
def features_and_target(df):
    #slices based on columns
    f = df.iloc[0 : , 1 : 31]
    #retrieves target column
    t = df["target"]
    return f, t

In [17]:

features, target= features_and_target(to_feed)

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

X_train, X_test, y_train, y_test = train_test_split(features,target, random_state=3000)
model = GaussianNB().fit(X=X_train, y=y_train)
print(str(model))
#predicts accuracy
accuracy = model.score(X_test, y_test)
#print statement and formatting
print(accuracy)

GaussianNB(priors=None, var_smoothing=1e-09)
0.6701807228915663


In [24]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor

def RFE_feature_selection():
    select = RFE(DecisionTreeRegressor(random_state = 3000), n_features_to_select = 4)
    
    #fit the RFE selector to the training data
    select.fit(X_train, y_train)

    #transform training and testing sets so only the selected features are retained
    X_train_selected = select.transform(X_train)
    X_test_selected = select.transform(X_test)

    model = GaussianNB().fit(X=X_train_selected, y=y_train)
    
    accuracy = model.score(X_test_selected, y_test)
    
    print("Selected features after RFE:")
    for i in range(0, len(select.get_support())):
        if select.get_support()[i] == True:
            print("\t" + str(features.columns[i]))
    
    print("DecisionTreeClassifier() Regression performance with selected features:")
    print(accuracy)

RFE_feature_selection()

Selected features after RFE:
	2
	7
	17
	18
DecisionTreeClassifier() Regression performance with selected features:
0.5941265060240963


In [19]:
feat_tuple = ["Reach","Age","SLpM","SApM","Str Def","TD Avg","TD Acc","TD Def","Sub Avg","KO/TKO Wins","SUB Wins","U-DEC Wins",
             "KO/TKO Losses", "SUB Losses", "U-DEC Losses"]
def predict_fight(fighter1, fighter2):
    tup1 = []
    tup2 = []
    for feat in feat_tuple:
        tup1.append(fighter_info_final[feat][fighter1])
        
    for feat in feat_tuple:
        tup1.append(fighter_info_final[feat][fighter2])
        tup2.append(fighter_info_final[feat][fighter2])
        
    for feat in feat_tuple:
        tup2.append(fighter_info_final[feat][fighter1])
    
    result1 = model.predict([tup1])[0]
    result2 = model.predict([tup2])[0]
    if(result1 == result2):
        return "Not Sure. Close Fight."
    elif(result1 == 1):
        return fighter1
    else:
        return fighter2

In [20]:
fighter_info_final.index = fighter_info_final["Name"]

In [21]:
predict_fight("Conor McGregor","Khabib Nurmagomedov")

'Khabib Nurmagomedov'